In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/openai-to-z-challenge/README


In [2]:
ref = pd.read_csv('/kaggle/input/openai-to-z-challenge/README')
print(ref)

               Welcome to the OpenAI to Z Challenge!
0      This is a Hackathon with no provided dataset.
1  Please refer to kaggle.com/competitions/openai...


In [3]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 17.3 MB/s eta 0:00:0000:01
  Attempting uninstall: openai
    Found existing installation: openai 1.70.0
    Uninstalling openai-1.70.0:
      Successfully uninstalled openai-1.70.0


In [ ]:
import openai as op

In [7]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("legolas_API")

In [ ]:
# import os
# from openai import OpenAI
# from kaggle_secrets import UserSecretsClient

# # Access OpenAI key from Kaggle Secrets
# user_secrets = UserSecretsClient()
# openai_key = user_secrets.get_secret("OpenAI")  # "OpenAI" should be the secret label

# # Create OpenAI client
# client = OpenAI(api_key=openai_key)

# # Request a research plan
# chat_completion = client.chat.completions.create(
#     model="gpt-4",
#     messages=[
#         {"role": "system", "content": "You are a world-class remote sensing archaeologist and research strategist."},
#         {"role": "user", "content": """
# You are participating in a global challenge to find unknown archaeological sites in the Amazon using satellite imagery, LIDAR, and historical documents.

# Your tools include:
# - High-resolution satellite imagery (Sentinel-2, Landsat, NICFI)
# - LIDAR data (1m–5m resolution) from OpenTopography
# - Colonial-era texts and Indigenous maps (to be parsed with GPT/NLP)
# - Access to geospatial libraries (rasterio, geopandas, openCV)
# - Python, Jupyter notebooks, Kaggle environment
# - Models: CNN, ViT, GPT-4, o4-mini

# The geographic focus is the Santarém region in northern Brazil. You're expected to:
# 1. Detect potential archaeological settlements hidden beneath forest canopy.
# 2. Validate coordinates with two independent methods (e.g., LIDAR + text).
# 3. Package your findings in a reproducible notebook + PDF report.

# Now, devise a **step-by-step research plan** including:
# - Data acquisition and bounding box
# - Data preprocessing and masking
# - Feature engineering and ML approach
# - Validation strategy
# - Expected output format
# - Risks and mitigation

# Please format the output as markdown with numbered steps.
# """}
#     ]
# )

# # Print the markdown-formatted research plan
# print(chat_completion.choices[0].message.content)

# Academic references

In [8]:
import requests

# --- 1. Источники из Starter Pack (вручную добавленные)
starter_sources = [
    "https://www.nature.com/articles/s41586-022-04780-4",  # Nature, 2022
    "https://peerj.com/articles/15137/",                    # PeerJ, 2023
]

print("Manual Sources from Starter Pack:\n")
for i, url in enumerate(starter_sources, 1):
    print(f"{i}. {url}")
print("\n" + "-"*70 + "\n")


# --- 2. Поисковые запросы по теме Amazon + LiDAR + Archaeology
queries = [
    "Amazon LiDAR archaeology site remote sensing Brazil",
    "Amazon Lost City LiDAR Sentinel-2 Landsat archaeology",
    "Amazonia geoglyphs LiDAR satellite archaeology",
]

# Semantic Scholar API базовый URL
base_url = "https://api.semanticscholar.org/graph/v1/paper/search"

# Для каждого запроса выполняем поиск и выводим статьи
for q_index, query in enumerate(queries, start=1):
    print(f"Query {q_index}: \"{query}\"\n")
    url = f"{base_url}?query={query}&limit=5&fields=title,authors,year,abstract,url"

    response = requests.get(url)
    data = response.json()

    for i, paper in enumerate(data.get("data", []), start=1):
        print(f"{i}. {paper['title']}")
        print("   Authors:", ", ".join(a["name"] for a in paper["authors"]))
        print("   Year:", paper["year"])
        print("   URL:", paper["url"])
        print()

    print("-" * 70 + "\n")


Manual Sources from Starter Pack:

1. https://www.nature.com/articles/s41586-022-04780-4
2. https://peerj.com/articles/15137/

----------------------------------------------------------------------

Query 1: "Amazon LiDAR archaeology site remote sensing Brazil"

1. Archaeological site segmentation of ancient city walls based on deep learning and LiDAR remote sensing
   Authors: Shunli Wang, Qingwu Hu, Shaohua Wang, M. Ai, P. Zhao
   Year: 2024
   URL: https://www.semanticscholar.org/paper/323feed7115c877f707b1694e0967cb70c253400

2. Tackling the Thorny Dilemma of Mapping Southeastern Sicily's Coastal Archaeology Beneath Dense Mediterranean Vegetation: A Drone‐Based LiDAR Approach
   Authors: Dario Calderone, Nicola Lercari, Davide Tanasi, Dennis Busch, Ryan Hom, R. Lanteri
   Year: 2024
   URL: https://www.semanticscholar.org/paper/ebfa660a1c44584cb0d9e6ddc2a2504a666049ae

3. A biomass map of the Brazilian Amazon from multisource remote sensing
   Authors: J. Ometto, E. Gorgens, Franci

In [9]:
import requests

# Тематические поисковые запросы
queries = [
    "Amazon LiDAR archaeology site remote sensing Brazil",
    "Amazon Lost City LiDAR Sentinel-2 Landsat archaeology",
    "Amazonia geoglyphs LiDAR satellite archaeology",
]

# Semantic Scholar API URL
base_url = "https://api.semanticscholar.org/graph/v1/paper/search"

# Список для хранения литературы
literature = []

for q_index, query in enumerate(queries, start=1):
    print(f"🔍 Query {q_index}: \"{query}\"\n")
    url = f"{base_url}?query={query}&limit=5&fields=title,authors,year,abstract,url,externalIds"

    response = requests.get(url)
    data = response.json()

    for i, paper in enumerate(data.get("data", []), start=1):
        title = paper.get("title", "N/A")
        authors = ", ".join(a.get("name", "") for a in paper.get("authors", []))
        year = paper.get("year", "N/A")
        abstract = paper.get("abstract", "No abstract available.")
        link = paper.get("url", "N/A")
        doi = paper.get("externalIds", {}).get("DOI", None)
        doi_link = f"https://doi.org/{doi}" if doi else "N/A"

        print(f"{i}. {title} ({year})")
        print(f"   Authors: {authors}")
        print(f"   DOI: {doi_link}")
        print(f"   URL: {link}")
        print(f"   Abstract: {abstract[:300]}{'...' if len(abstract) > 300 else ''}\n")

        literature.append({
            "title": title,
            "authors": authors,
            "year": year,
            "abstract": abstract,
            "url": link,
            "doi": doi_link
        })

    print("-" * 80 + "\n")

# Финальный блок литературы
print("📚 Final Literature List (with DOI & Abstract):\n")
for i, ref in enumerate(literature, 1):
    print(f"{i}. {ref['title']} ({ref['year']})")
    print(f"   Authors: {ref['authors']}")
    print(f"   DOI: {ref['doi']}")
    print(f"   Link: {ref['url']}")
    print(f"   Abstract: {ref['abstract'][:300]}{'...' if len(ref['abstract']) > 300 else ''}\n")


🔍 Query 1: "Amazon LiDAR archaeology site remote sensing Brazil"

--------------------------------------------------------------------------------

🔍 Query 2: "Amazon Lost City LiDAR Sentinel-2 Landsat archaeology"

--------------------------------------------------------------------------------

🔍 Query 3: "Amazonia geoglyphs LiDAR satellite archaeology"

1. Geometry by Design: Contribution of Lidar to the Understanding of Settlement Patterns of the Mound Villages in SW Amazonia (2020)
   Authors: José Iriarte, M. Robinson, J. G. Souza, A. Damasceno, Franciele da Silva, Francisco Ruji Nakahara, A. Ranzi, L. Aragão
   DOI: https://doi.org/10.5334/jcaa.45
   URL: https://www.semanticscholar.org/paper/d4d834bdbffd7eec8d668debc7d2957c6c84b927
   Abstract: Recent research has shown that the entire southern rim of Amazonia was inhabited by earth-building societies involving landscape engineering, landscape domestication and likely low-density urbanism during the Late Holocene. However, the 

In [13]:
import requests

# Ключевые слова для фильтрации аннотаций
keywords = ["geoglyph", "earthworks", "ditch"]

# Тематические поисковые запросы
queries = [
    "Amazon LiDAR archaeology site remote sensing Brazil",
    "Amazon Lost City LiDAR Sentinel-2 Landsat archaeology",
    "Amazonia geoglyphs LiDAR satellite archaeology",
]

# Semantic Scholar API базовый URL
base_url = "https://api.semanticscholar.org/graph/v1/paper/search"

# Собираем отфильтрованные статьи
filtered_papers = []

for query in queries:
    url = f"{base_url}?query={query}&limit=10&fields=title,authors,year,abstract,url,externalIds"
    response = requests.get(url)
    if response.status_code != 200:
        continue
    data = response.json()
    
    for paper in data.get("data", []):
        abstract = paper.get("abstract")
        if abstract and any(kw in abstract.lower() for kw in keywords):
           title = paper.get("title", "N/A")
           authors = ", ".join(a.get("name", "") for a in paper.get("authors", []))
           year = paper.get("year", "N/A")
           url = paper.get("url", "")
           doi = paper.get("externalIds", {}).get("DOI", None)
           doi_link = f"[{doi}](https://doi.org/{doi})" if doi else "N/A"
           filtered_papers.append({
               "Title": f"[{title}]({url})",
               "Authors": authors,
               "Year": year,
               "DOI": doi_link,
               "Abstract": (abstract[:200] + "...") if len(abstract) > 200 else abstract
                })


# Генерация Markdown-таблицы
def generate_markdown_table(papers):
    if not papers:
        return "No relevant papers found."

    headers = ["Title", "Authors", "Year", "DOI", "Abstract (truncated)"]
    md = "| " + " | ".join(headers) + " |\n"
    md += "| " + " | ".join(["---"] * len(headers)) + " |\n"

    for paper in papers:
        row = [paper["Title"], paper["Authors"], str(paper["Year"]), paper["DOI"], paper["Abstract"].replace("\n", " ")]
        md += "| " + " | ".join(row) + " |\n"
    return md

# Вывод Markdown-таблицы
markdown_output = generate_markdown_table(filtered_papers)
print(markdown_output)

| Title | Authors | Year | DOI | Abstract (truncated) |
| --- | --- | --- | --- | --- |
| [Geometry by Design: Contribution of Lidar to the Understanding of Settlement Patterns of the Mound Villages in SW Amazonia](https://www.semanticscholar.org/paper/d4d834bdbffd7eec8d668debc7d2957c6c84b927) | José Iriarte, M. Robinson, J. G. Souza, A. Damasceno, Franciele da Silva, Francisco Ruji Nakahara, A. Ranzi, L. Aragão | 2020 | [10.5334/jcaa.45](https://doi.org/10.5334/jcaa.45) | Recent research has shown that the entire southern rim of Amazonia was inhabited by earth-building societies involving landscape engineering, landscape domestication and likely low-density urbanism du... |



In [18]:
import folium
from folium.plugins import MarkerCluster

# Центр Амазонии (пример: Xingu River region)
center_lat, center_lng = -10.0, -62.0

# Создаём карту

# Или
m = folium.Map(location=[center_lat, center_lng], zoom_start=5, tiles="OpenStreetMap")


# Добавляем кластер маркеров
marker_cluster = MarkerCluster().add_to(m)

# Примеры координат найденных объектов
coordinates = [
    {"lat": -10.1234, "lng": -62.4321, "label": "Possible Earthwork Site"},
    {"lat": -8.5678, "lng": -60.9876, "label": "LiDAR anomaly"},
]

# Добавляем маркеры
for point in coordinates:
    folium.Marker(
        location=[point["lat"], point["lng"]],
        popup=point["label"],
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(marker_cluster)

# Показываем карту прямо в Jupyter Notebook




m

In [19]:
import folium
from folium import GeoJson
from folium.plugins import MarkerCluster

# Центр карты — Амазония
center_lat, center_lng = -7.0, -63.0
m = folium.Map(location=[center_lat, center_lng], zoom_start=5, tiles="CartoDB positron")

# === 1. Границы Амазонского биома (упрощённый полигон) ===
# Пример границ — сильно упрощён для демонстрации
amazon_polygon = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [-74, -5], [-70, 2], [-60, 5], [-50, 1],
                [-45, -3], [-50, -10], [-55, -15],
                [-60, -12], [-65, -10], [-70, -8],
                [-74, -5]  # замыкаем
            ]]
        },
        "properties": {"name": "Amazon Biome"}
    }]
}

folium.GeoJson(amazon_polygon, name="Amazon Biome",
               style_function=lambda x: {
                   "fillColor": "green",
                   "color": "darkgreen",
                   "weight": 2,
                   "fillOpacity": 0.1
               }).add_to(m)

# === 2. Точки из литературы ===
literature_points = [
    {"name": "Santarém", "desc": "Pre-colonial urbanism (2025)", "coords": [-2.443, -54.708]},
    {"name": "Xingu Headwaters (Kuhikugu)", "desc": "Lost City of Z", "coords": [-11.5, -53.0]},
    {"name": "Acre", "desc": "Geoglyphs region (2022)", "coords": [-9.5, -67.5]},
    {"name": "Llanos de Mojos", "desc": "Low-density cities (2022)", "coords": [-14.0, -65.0]},
    {"name": "Santarém Region", "desc": "Human settlement patterns (2018)", "coords": [-2.5, -55.0]},
]

# Кластер маркеров
marker_cluster = MarkerCluster().add_to(m)

for point in literature_points:
    folium.Marker(
        location=point["coords"],
        popup=f"<b>{point['name']}</b><br>{point['desc']}",
        icon=folium.Icon(color="green", icon="book")
    ).add_to(marker_cluster)

# Слои и отображение
folium.LayerControl().add_to(m)
m
